In [2]:
#!/usr/bin/python
# coding: utf-8
from __future__ import print_function

__source__ = "lsi_classification1,ipynb"
__author__ = "Frank J. Greco"
__copyright__ = "Copyright 2015-2018, Frank J. Greco"
__credits__ = []
__license__ = "Apache"
__version__ = "1.0.1"
__email__ = ""
__status__ = "Development"


# Build an LSI model on set of transcript testfiles
# Perform a similarity query  of a testfile against the model
# <provide outside reference here>


In [3]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload
import sys
sys.path.append('/Users/fjgreco/Dev-Atlas')

In [4]:
import logging
import sys
import os
import re
from gensim import corpora, models, similarities
from collections import defaultdict
from preprocess import create_doc_index
from preprocess import TagWorksheet


#reload(sys)
#sys.setdefaultencoding('utf8')

In [5]:


xls_pathname = '../CC_FILES_TAGLIST/TW Case List.xlsx'

tw=TagWorksheet(xls_pathname)


*** Begin process_worksheet ***

[u'CASE IDS', u'Sheet1', u'caseannotation-20170613T220819']
Sort Tag Dictionary
0 ['NaN']
1 [u'TW06', u'TW08', u'TW15', u'TW24']
2 [u'TW02', u'TW05', u'TW09', u'TW10', u'TW13']
3 [u'TW01', u'TW08', u'TW13', u'TW15', u'TW17', u'TW24TW06']
4 [u'TW01', u'TW13', u'TW17', u'TW24']
5 [u'TW01', u'TW15', u'TW16', u'TW24']
6 [u'TW03', u'TW05', u'TW06', u'TW08', u'TW16', u'TW24']
9 [u'TW01', u'TW02', u'TW04', u'TW05', u'TW11', u'TW13', u'TW15', u'TW16', u'TW17', u'TW18', u'TW19', u'TW24']
10 [u'TW01', u'TW02', u'TW11', u'TW13', u'TW16', u'TW17', u'TW18']
12 [u'TW01', u'TW02', u'TW05', u'TW06', u'TW17', u'TW24TW06']
14 [u'TW01', u'TW02', u'TW05', u'TW06', u'TW16', u'TW19', u'TW24']
20 [u'TW08']
21 [u'TW02', u'TW05', u'TW09', u'TW24']
22 [u'TW02', u'TW08', u'TW11']
24 [u'TW01', u'TW03', u'TW04', u'TW06', u'TW11', u'TW17', u'TW24']
26 [u'TW02', u'TW09', u'TW24']
27 [u'TW02', u'TW05', u'TW08', u'TW24']
28 [u'TW01', u'TW03', u'TW04', u'TW05', u'TW10', u'TW11', u'TW17

10 TW01,TW02,TW11,TW13,TW16,TW17,TW18
12 TW01,TW02,TW05,TW06,TW17,TW24TW06
14 TW01,TW02,TW05,TW06,TW16,TW19,TW24
20 TW08
21 TW02,TW05,TW09,TW24
22 TW02,TW08,TW11
24 TW01,TW03,TW04,TW06,TW11,TW17,TW24
26 TW02,TW09,TW24
27 TW02,TW05,TW08,TW24
28 TW01,TW03,TW04,TW05,TW10,TW11,TW17,TW20,TW24TW17
29 TW01,TW02,TW03,TW05,TW16,TW24
31 TW01,TW06,TW08
32 TW03,TW06,TW17
33 TW02,TW04,TW06,TW09,TW21,TW24
34 TW02,TW07,TW09,TW24
40 TW01,TW06,TW07,TW13,TW15,TW24
41 TW10
42 TW02
43 TW02,TW08,TW09
45 TW02,TW05,TW08,TW09,TW15,TW22,TW24,TW24TW09
46 TW01,TW03,TW04,TW06,TW11,TW13,TW16
48 TW01,TW02,TW11,TW17,TW19
50 TW13
51 TW02,TW09,TW17,TW19,TW24,TW24TW09,TW24TW12
52 TW06,TW08,TW13,TW15,TW16,TW17,TW24TW07,TW24TW08
53 TW02,TW07,TW09,TW10,TW11,TW15,TW17,TW18
55 TW01,TW04,TW06,TW11,TW13,TW17
60 TW01,TW08,TW18,TW24,TW24TW08
61 TW01,TW02,TW05,TW08,TW15,TW18,TW21,TW24,TW24TW08
64 TW01,TW02,TW04,TW06,TW11,TW16,TW19
65 TW01,TW04,TW06,TW07,TW13,TW15,TW20
66 TW02,TW06
68 TW01,TW02,TW04,TW05,TW09,TW13,TW16,TW17,TW24


In [6]:
transcript_dir='../CC_TRANSCRIPTS3'
print ('transcript_dir:', transcript_dir)

transcript_dir: ../CC_TRANSCRIPTS3


In [7]:

documents, doc_index= create_doc_index(transcript_dir)

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())

texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

# remove words that appear only once

frequency = defaultdict(int)

for text in texts:
             for token in text:
                 frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

#Create and save dictionary
dictionary = corpora.Dictionary(texts)
dictionary.save('./tmp/TopicClassifier.dict')

print ("Create and save dictionary")
#print(dictionary)
#print(dictionary.token2id)

Create and save dictionary


In [8]:
testfile1='../CC_TRANSCRIPTS3/Copy of 825 SCI EA 825 edited.srt.txt'
testfile2='../CC_TRANSCRIPTS3/Copy of 001 ENLEAYA001.srt.txt'
testfile=testfile1

print ('Open testfile:', testfile)

tf = open(testfile, 'r')
new_doc=(str.decode(tf.read(), "UTF-8", "ignore"))
tf.close()

Open testfile: ../CC_TRANSCRIPTS3/Copy of 825 SCI EA 825 edited.srt.txt


In [9]:
print ("Query (testfile) document: ")
print(new_doc)

Query (testfile) document: 
In case you think there's something missing, and then we'll see what you got.
Okay.
The tongue does with the .
The tongue? Oh yeah, because the trachea will swallow it. No, the trachea's the air....
The esophagus close your...
The esophagus is the food pipe for the thing.
No, the esophagus is the thing that stops the food...
The esophagus is the food part, the windpipe is the trachea.
You talking about the epiglottis.
Oh, the epiglottis!
That's the flap of skin that stops it, food going down your trachea, your air passageway.
So you don't choke.
Right.
How do you spell that?
How do you spell what?
Are you gonna put that in there somewhere? Well, here's what I'm talking about, you only need four people in this group, and you don't have as many pieces of paper. Don't take more without asking, why are you taking more? Why?
Because Tyler got  on it.
Go wash it in the sink. I'll mark down on my clipboard who's having trouble with self-control. Go to the sink and 

In [11]:
print ("Vector representation of query (testfile) document:")
new_vec = dictionary.doc2bow(new_doc.lower().split())

Vector representation of query (testfile) document:


In [12]:
print(new_vec[0:10])

[(1, 4), (3, 1), (5, 36), (6, 3), (9, 1), (12, 4), (16, 14), (20, 10), (22, 2), (26, 18)]


In [13]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('./tmp/TopicClassifier.mm', corpus) # store to disk, for later use

In [16]:
print ("Create corpus from source text")
for item in corpus[0:10]:
    print (item[0:10])

Create corpus from source text
[(0, 1), (1, 13), (2, 1), (3, 5), (4, 1), (5, 4), (6, 3), (7, 1), (8, 1), (9, 1)]
[(1, 1), (5, 4), (12, 1), (20, 2), (22, 1), (24, 1), (26, 5), (45, 3), (49, 3), (50, 1)]
[(1, 2), (5, 1), (9, 1), (16, 3), (20, 3), (26, 2), (30, 1), (32, 1), (37, 1), (45, 2)]
[(1, 33), (5, 3), (6, 1), (7, 2), (12, 5), (16, 3), (18, 1), (20, 18), (21, 1), (22, 6)]
[(5, 1), (28, 1), (72, 2), (94, 1), (105, 1), (107, 2), (115, 1), (136, 1), (142, 2), (145, 3)]
[(1, 6), (3, 1), (5, 3), (6, 1), (12, 3), (16, 2), (20, 2), (23, 1), (26, 2), (30, 15)]
[(1, 12), (2, 1), (3, 1), (5, 8), (6, 5), (9, 2), (10, 3), (11, 1), (12, 4), (16, 18)]
[(1, 4), (5, 27), (12, 2), (16, 12), (18, 8), (20, 6), (22, 5), (26, 16), (30, 4), (45, 1)]
[(1, 14), (2, 2), (3, 2), (5, 9), (7, 7), (9, 5), (12, 3), (16, 9), (18, 1), (20, 10)]
[(1, 25), (2, 4), (5, 19), (6, 2), (9, 1), (12, 5), (16, 26), (18, 4), (20, 14), (22, 5)]


In [18]:
lsi = models.LdaModel(corpus, id2word=dictionary, num_topics=10)

# Transform testfile document to bag of words
vec_bow = new_vec
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print("query vector:",vec_lsi)

index = similarities.MatrixSimilarity(lsi[corpus]) # transform corpus to LSI space and index it

#index = similarities.MatrixSimilarity.load(save_index)

query vector: [(1, 0.5415484), (6, 0.032529317), (7, 0.015800381), (8, 0.17949344), (9, 0.22143927)]


In [25]:
print("Creating sims...")
sims = index[vec_lsi] # perform a similarity query against the corpus
## print (document_number, document_similarity) 2-tuples
#for sim in list(enumerate(sims)):
#    print (sim)

print("Sorting sims...")
sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
# print sorted (document number, similarity score) 2-tuples
for sim in sims_sorted:
    print (sim)

Creating sims...
Sorting sims...
(138, 0.99273556)
(176, 0.9900455)
(37, 0.9877405)
(211, 0.97048485)
(107, 0.95010936)
(11, 0.93996584)
(89, 0.93889475)
(27, 0.93707967)
(43, 0.92605704)
(72, 0.92357326)
(35, 0.92195594)
(80, 0.92097265)
(32, 0.9201813)
(146, 0.9185483)
(152, 0.9180107)
(95, 0.90139097)
(205, 0.89727396)
(238, 0.89358133)
(93, 0.88729584)
(166, 0.8840693)
(12, 0.88336104)
(51, 0.88336104)
(130, 0.88336104)
(133, 0.88336104)
(158, 0.88336104)
(206, 0.88336104)
(236, 0.88336104)
(203, 0.8817205)
(15, 0.88117623)
(242, 0.8677536)
(226, 0.86386806)
(104, 0.8509684)
(46, 0.8462578)
(84, 0.8303169)
(109, 0.80592746)
(99, 0.7982383)
(231, 0.7718638)
(48, 0.767689)
(184, 0.7461103)
(139, 0.7273067)
(38, 0.72094274)
(232, 0.7169068)
(142, 0.70070446)
(113, 0.70038545)
(222, 0.69911623)
(132, 0.6564579)
(151, 0.63809526)
(0, 0.62806344)
(110, 0.6233275)
(17, 0.6104791)
(141, 0.5715492)
(212, 0.5594895)
(217, 0.55724263)
(195, 0.55367625)
(68, 0.5430868)
(94, 0.54103047)
(49, 0.

In [26]:
print("Doc Index...")
for item in doc_index:
    print (item)


Doc Index...
(0, '118', 'Copy of 118 GEN MC 118edited.srt.txt')
(1, '014 ', 'Copy of 014B LMECYA014B.srt.txt')
(2, '285 ', 'Copy of 285B SCI EA 285 B edited.srt.txt')
(3, '161', 'Copy of 161 SCI EA 161 edited.srt.txt')
(4, '085 ', 'Copy of 085A MUSIC EMC 085 A.srt.txt')
(5, '074 ', 'Copy of 074A ENL EMC 074A edited.srt.txt')
(6, '101', 'Copy of 101 SSH EA 101 edited.srt.txt')
(7, '079', 'Copy of 079 MUSIC EMC 079.srt.txt')
(8, '1047', 'Copy of 1047 SCI AYA 1047edited.srt.txt')
(9, '128', 'Copy of 128 PE EAYA 128edited.srt.txt')
(10, '244 ', 'Copy of 244A SCI EA 244 A edited.srt.txt')
(11, '243', 'Copy of 243 SCI EA 243 edited.srt.txt')
(12, '150', 'Copy of 150 GEN EC 150 edited.srt.txt')
(13, '1360', 'Copy of 1360 GEN_EC_1360_Video_Recording_A_edited.srt.txt')
(14, '618', 'Copy of 618 GEN MC 618 edited.srt.txt')
(15, '165', 'Copy of 165 CTE EAYA 165 edited.srt.txt')
(16, '1082', 'Copy of 1082 SCI AYA 1082-2edited.srt.txt')
(17, '154 ', 'Copy of 154B CTE EAYA 154 B.srt.txt')
(18, '1015'

In [30]:
print("Combined Results...")
#[ print ('Seq: {} CaseID: {} File: {} Similarity: {}'.format(item[0][0], item[0][1], item[0][2], item[1])) for item in zip(doc_index,sims)]

print ("Len(sims): {}  Len(Index): {}".format(len(sims), len(doc_index)))

Combined Results...
Len(sims): 243  Len(Index): 243


In [28]:
combined=[]
for item in zip(doc_index, sims):
    combined.append((item[0][1], item[0][2], item[1]))

combined_sorted=sorted(combined, key=lambda x: -x[-1])

for item in combined_sorted:
    print(item)

('825', 'Copy of 825 SCI EA 825 edited.srt.txt', 0.99273556)
('1838', 'Copy of 1838 SCI_EA_1838_Video_Recording_A_edited.srt.txt', 0.9900455)
('181', 'Copy of 181 ELA EA 181.srt.txt', 0.9877405)
('611', 'Copy of 611 GEN MC 611 edited.srt.txt', 0.97048485)
('066', 'Copy of 066 ENS ECYA 066 edited.srt.txt', 0.95010936)
('243', 'Copy of 243 SCI EA 243 edited.srt.txt', 0.93996584)
('134', 'Copy of 134 MATH AYA 134 edited.srt.txt', 0.93889475)
('050', 'Copy of 050 GEN EC 050edited.srt.txt', 0.93707967)
('804', 'Copy of 804 MATH EA 804 edited.srt.txt', 0.92605704)
('010', 'Copy of 010 SCECYA010.srt.txt', 0.92357326)
('004 ', 'Copy of 004A MUSICEAYA004A.srt.txt', 0.92195594)
('135', 'Copy of 135 SC ECYA 135 edited.srt.txt', 0.92097265)
('012 ', 'Copy of 012B PE EAYA 012Bedited.srt.txt', 0.9201813)
('207', 'Copy of 207 LRLA EMC 207edited.srt.txt', 0.9185483)
('828', 'Copy of 828 SCI EA 828 edited.srt.txt', 0.9180107)
('129', 'Copy of 129 LRLA EMC 129edited.srt.txt', 0.90139097)
('158', 'Copy o

In [42]:
s = []
a=[]
print ("Create list of the top 10 similar documents; exclude first item as this is the test file")
for item in combined_sorted[1:11]:
    taglist=tw.get_tags(int(item[0]))
    print(item,'\n',taglist)
    s.append(set(taglist))
    a.extend(taglist)
 

Create list of top 10 similar documents; exclude first item as this is the test file
('1838', 'Copy of 1838 SCI_EA_1838_Video_Recording_A_edited.srt.txt', 0.9900455) 
 [u'TW01', u'TW02', u'TW03', u'TW05', u'TW08', u'TW13', u'TW16', u'TW17', u'TW22', u'TW24TW08']
('181', 'Copy of 181 ELA EA 181.srt.txt', 0.9877405) 
 [u'TW01', u'TW08']
('611', 'Copy of 611 GEN MC 611 edited.srt.txt', 0.97048485) 
 [u'TW02', u'TW09m', u'TW24TW11']
('066', 'Copy of 066 ENS ECYA 066 edited.srt.txt', 0.95010936) 
 [u'TW02', u'TW06']
('243', 'Copy of 243 SCI EA 243 edited.srt.txt', 0.93996584) 
 [u'TW01', u'TW02', u'TW04', u'TW05', u'TW08', u'TW24', u'TW24TW08']
('134', 'Copy of 134 MATH AYA 134 edited.srt.txt', 0.93889475) 
 [u'TW02', u'TW05', u'TW09', u'TW15', u'TW22', u'TW23', u'TW24', u'TW24TW17']
('050', 'Copy of 050 GEN EC 050edited.srt.txt', 0.93707967) 
 [u'TW13']
('804', 'Copy of 804 MATH EA 804 edited.srt.txt', 0.92605704) 
 [u'TW01', u'TW08m', u'TW11', u'TW23', u'TW24TW07']
('010', 'Copy of 010 SC

In [49]:

y = set.intersection(*s)

print ("Tag set intersection:" , y)

Tag set intersection: set([])


In [54]:
import collections

counter=collections.Counter(a)

#print(counter.values())
#print(counter.keys())
print (zip(counter.keys(), counter.values()))

[(u'TW24TW07', 1), (u'TW23', 2), (u'TW09m', 1), (u'TW22', 2), (u'TW24TW08', 2), (u'TW06', 1), (u'TW05', 3), (u'TW04', 1), (u'TW03', 1), (u'TW02', 6), (u'TW01', 6), (u'TW24', 3), (u'TW09', 1), (u'TW08', 3), (u'TW24TW11', 1), (u'TW24TW17', 1), (u'TW15', 1), (u'TW16', 2), (u'TW17', 3), (u'TW11', 2), (u'TW13', 4), (u'TW18', 1), (u'TW08m', 1)]


In [56]:
print ('Candidate Tags:')
for item in counter.most_common(10):
    print (item)

Candidate Tags:
(u'TW02', 6)
(u'TW01', 6)
(u'TW13', 4)
(u'TW05', 3)
(u'TW24', 3)
(u'TW08', 3)
(u'TW17', 3)
(u'TW23', 2)
(u'TW22', 2)
(u'TW24TW08', 2)


In [24]:
# TW24,TW23,TW22,TW13,TW08,TW01,TW17,TW02
